In [ ]:
import os
import json

import numpy as np
from scipy.spatial.transform import Rotation as R
import pandas as pd
import matplotlib.pyplot as plt

import tools

# Pandas to display 3 decimal points
pd.options.display.float_format = '{:.3f}'.format
# Make matplotlib interractive in jupyter
%matplotlib widget

In [ ]:
def plot_est(state_df, sample_rate, title):
    t = np.arange(len(state_df))/sample_rate
    # Plot the estimated locations, and angles over time, with time axis in seconds, in 2 subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 4))
    fig.suptitle(title)
    ax1.plot(t, state_df['x_w'], 'r', label='x')
    ax1.plot(t, state_df['y_w'], 'g', label='y')
    ax1.plot(t, state_df['z_w'], 'b', label='z')
    ax1.legend(loc='upper left')
    ax1.set_ylabel('Location (m)')
    ax2.plot(t, state_df['pi_w']/(2*np.pi)*360, 'r', label='pitch')
    ax2.plot(t, state_df['ro_w']/(2*np.pi)*360, 'g', label='roll')
    ax2.plot(t, state_df['ya_w']/(2*np.pi)*360, 'b', label='yaw')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Angle (deg)')
    plt.tight_layout()
    plt.show()


def run_location_tests(accelerometer, gyroscope, sample_rate, trackers: dict, plot_distance=True, plot_time=False):
    for i in range(len(accelerometer)):
        for tracker_name in trackers:
            trackers[tracker_name].add_measurement(accelerometer[i], gyroscope[i])

    df_list = [trackers[tracker_name].get_state_history_df() for tracker_name in trackers]

    if plot_distance:
        # Plot the estimated locations over time, with time axis in seconds
        plt.figure()
        for tracker_name, df in zip(trackers, df_list):
            plt.plot(df['x_w'].to_numpy(), df['y_w'].to_numpy(), label=tracker_name)
        plt.legend(loc='upper left')
        plt.xlabel('x position (m)')
        plt.ylabel('y position (m)')
        plt.tight_layout()
        plt.show()

    if plot_time:
        for tracker_name, df in zip(trackers, df_list):
            plot_est(df, sample_rate, tracker_name)

def test_file(file, calibration, plot_time=False, plot_distance=False, plot_data=False, settle_time=3, zero_start=False):
    accelerometer, gyroscope, sample_rate, temperature, temperature_sample_rate = tools.read_data_file(
        os.path.join('..', 'tmp', file), lowpass=150, settle_time=settle_time
    )
    # Implement calibration
    accelerometer, gyroscope = tools.apply_calibration(accelerometer, gyroscope, temperature, calibration)

    # Get initial rotation
    init_r = tools.Location3d.initial_rotation_from_accelerometer(accelerometer, sample_rate, length=0.5)
    init_pitch, init_roll = tools.Location3d.initial_rotation_from_accelerometer(accelerometer, sample_rate, length=0.5, as_rotation=False)
    # Zero the gyroscope
    if zero_start:
        zero_point = np.mean(gyroscope[:int(sample_rate*0.5)], axis=0)
        gyroscope -= zero_point

    if plot_data:
        tools.plot_data(accelerometer, gyroscope, sample_rate, f'{os.path.basename(file)}', temperature, temperature_sample_rate)
        tools.plot_data(init_r.apply(accelerometer), gyroscope, sample_rate, f'Rotated: {os.path.basename(file)}', temperature, temperature_sample_rate)

    # Process noise covariance (Too slow to react? Increase Q; Too sensitive to noise? Decrease the Q)
    config = tools.UkfConfig(
        state_n=len(tools.Location3d.state_names), measurement_n=6,
        alpha=1e-3, beta=2, kappa=0,
        Q=1e-4, P=1e-6, R=np.array(calibration['R']),
        init_r=init_r
    )
    # ukf_obj = tools.Location3dUkf(sample_rate=sample_rate, config=config)
    dr_obj = tools.Location3d(sample_rate=sample_rate, init_r=init_r)

    run_location_tests(
        # accelerometer, gyroscope, sample_rate, {'ukf': ukf_obj, 'dr': dr_obj}, plot_time=True
        accelerometer, gyroscope, sample_rate, {'dr': dr_obj}, plot_time=plot_time, plot_distance=plot_distance,
    )
    dr_df = dr_obj.get_state_history_df()
    final_x = dr_df.iloc[-1]["x_w"]
    final_y = dr_df.iloc[-1]["y_w"]
    final_z = dr_df.iloc[-1]["z_w"]

    # Calculate the integral of the rotated accelerometer vector to get an ideal velocity and distance
    rotated_accelerometer = init_r.apply(accelerometer)
    velocity = np.cumsum(rotated_accelerometer - np.array([0, 0, tools.GRAVITY]), axis=0) / sample_rate
    distance = np.cumsum(velocity, axis=0) / sample_rate

    return {
        'file': os.path.basename(file),
        'temperature': np.mean(temperature),
        'init_pitch': init_pitch,
        'init_roll': init_roll,
        'mag_a': np.mean(np.linalg.norm(accelerometer, axis=1)),
        'final_x': final_x,
        'final_y': final_y,
        'final_z': final_z,
        'distance': np.linalg.norm([final_x, final_y, final_z]),
        'distance_sum': np.linalg.norm(distance[-1]),
        'xy_distance': np.linalg.norm([final_x, final_y]),
        'xy_distance_sum': np.linalg.norm(distance[-1][0:2]),
    }

In [ ]:
# Read calibration data
file = 'calibration.json'
with open(file) as f:
    calibration = json.load(f)

In [ ]:
test_file('test_5v/move.json', calibration, plot_time=True, plot_distance=True, plot_data=True, settle_time=0.1)
# test_file('cal_5v/cal_31.json', calibration, plot_time=True, plot_distance=True, plot_data=True)

In [ ]:
cal_path = os.path.join('..', 'tmp', 'cal_5v')
# Get list of all JSON files in the path 
file_names = sorted([os.path.join(cal_path, f) for f in os.listdir(cal_path) if f.endswith('.json')])

df = []
for file in file_names:
    df.append(test_file(file, calibration, zero_start=False))
df = pd.DataFrame(df)

In [ ]:
# Get the cross correlation for temperature in the dataframe
corr = df.drop(columns=['file']).corr()
# Display sorted by absolute value
display(corr['temperature'].sort_values(key=lambda x: np.abs(x), ascending=False))

display(df)
display(df.describe())

# Display a historgram of the distance and the xy_distance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.hist(df['distance'], bins=20)
ax1.set_ylabel('Distance (m)')
ax2.hist(df['xy_distance'], bins=20)
ax2.set_xlabel('Distance (m)')
ax2.set_ylabel('XY Distance (m)')
plt.tight_layout()
plt.show()

# Display distance vs temperature
plt.figure()
plt.plot(df['temperature'], df['xy_distance'], 'o')
plt.xlabel('Temperature (C)')
plt.ylabel('XY Distance (m)')
plt.tight_layout()
plt.show()

In [ ]:
cal_path = os.path.join('..', 'tmp', 'test_5v')
# Get list of all JSON files in the path 
file_names = sorted([os.path.join(cal_path, f) for f in os.listdir(cal_path) if f.endswith('.json')])

test_df = []
for file in file_names:
    test_df.append(test_file(
        file, calibration, settle_time=0.1, plot_time=True, plot_distance=True, plot_data=True, zero_start=False))
test_df = pd.DataFrame(test_df)
display(test_df)